## CLUSTERING AS A PRE-PROCESSING STEP (k=30)-

#### Since the models directly done on the Kaggle dataset didn't perform well, we will do another clustering for the hotels and check the models on the newly clustered data.
***
### 1. Load the cleaned original train dataset-

In [1]:
import pandas as pd 
import numpy as np 

path_train_final ='F:/AML/Project/clean dataset/FinalExpedia_set_cleaned/Train_set_cleaned.csv'

train_set=pd.read_csv(path_train_final)

In [2]:
train_set.shape

(2512931, 39)

### 2. Clustering-
#### Elbow method which we saw earlier gave optimum k=20.
***

##### KMeans algorithm is used for creating the optimum no.of clusters (k=20) and we have seen this performance already. Now, let us check with k=30 to understand how accuracy is with diff. cluster sizes.

In [3]:
from sklearn.cluster import KMeans

Kmean = KMeans(n_clusters=30).fit(train_set[['srch_destination_id', 'srch_destination_type_id', 'cnt',
       'hotel_continent', 'hotel_country', 'hotel_market']])

new_clusters=Kmean.labels_
train_set["new_hotel_cluster"]=new_clusters #update train_set with new hotel clusters
train_set["new_hotel_cluster"].value_counts() #new cluster distribution

C:\Users\User-\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


14    560489
24    332663
3     234935
0     129484
4     107417
18    105516
21    103194
7      97674
10     93873
6      74182
15     66945
20     63257
16     56621
13     54453
26     54090
28     48290
11     48065
1      44637
22     40353
17     40353
12     25946
2      25943
8      20583
25     14909
19     14550
5      14119
9      13276
23     10935
29     10206
27      5973
Name: new_hotel_cluster, dtype: int64

#### Split this train set to train and test datasets in 80:20 ratio.

In [4]:
predictors = [c for c in train_set.columns if c not in ["new_hotel_cluster"]]

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_set[predictors], train_set["new_hotel_cluster"], test_size=0.2)

### 3. Using models-
#### Apply the classification models on this clustered data.

#### RANDOM FOREST-
***
##### a. with 50 binary trees-

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

clf = RandomForestClassifier(n_estimators=50, min_weight_fraction_leaf=0.1)
clf.fit(X_train,y_train)
preds = clf.predict(X_test)

In [7]:
from sklearn.metrics import f1_score,accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:",precision_score(y_test, preds, average="micro"))
print("Recall:",recall_score(y_test, preds,  average="micro"))
print("F1 Score:",f1_score(y_test, preds,  average="micro"))

Accuracy: 0.4232660216042198
Precision: 0.4232660216042198
Recall: 0.4232660216042198
F1 Score: 0.4232660216042198


##### b. with 100 binary trees-

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

clf = RandomForestClassifier(n_estimators=100, min_weight_fraction_leaf=0.1)
clf.fit(X_train,y_train)
preds = clf.predict(X_test)

In [9]:
from sklearn.metrics import f1_score,accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:",precision_score(y_test, preds, average="micro"))
print("Recall:",recall_score(y_test, preds,  average="micro"))
print("F1 Score:",f1_score(y_test, preds,  average="micro"))

Accuracy: 0.42436831832100713
Precision: 0.42436831832100713
Recall: 0.42436831832100713
F1 Score: 0.42436831832100713


#### KERAS-
***
##### Load the required libraries-

In [10]:
!pip install tensorflow

In [11]:
!pip install scikeras
import tensorflow as tf
import keras
from scikeras.wrappers import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import KFold,train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

import os
import gc
import zipfile
import io

import pickle
import time
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

##### Consider the initial train_set-

In [12]:
train_set['new_hotel_cluster'].replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29],  
                               ['zero','One','Two','Three','Four','Five','Six','Seven','Eight','Nine','Ten','Eleven','Twelve','Thirteen','Fourteen','Fifteen','Sixteen','Seventeen','Eighteen','Nineteen','Twenty','Twenty one','Twenty two','Twenty three','Twenty four','Twenty five','Twenty six','Twenty seven','Twenty eight','Twenty nine'], 
                               inplace=True)
train_set['new_hotel_cluster'].value_counts()

Fourteen        560489
Twenty four     332663
Three           234935
zero            129484
Four            107417
Eighteen        105516
Twenty one      103194
Seven            97674
Ten              93873
Six              74182
Fifteen          66945
Twenty           63257
Sixteen          56621
Thirteen         54453
Twenty six       54090
Twenty eight     48290
Eleven           48065
One              44637
Twenty two       40353
Seventeen        40353
Twelve           25946
Two              25943
Eight            20583
Twenty five      14909
Nineteen         14550
Five             14119
Nine             13276
Twenty three     10935
Twenty nine      10206
Twenty seven      5973
Name: new_hotel_cluster, dtype: int64

In [13]:
X_train_k = train_set.drop(['new_hotel_cluster'], axis=1)
y_train_k = train_set['new_hotel_cluster']

In [14]:
from sklearn.preprocessing import LabelEncoder

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train_k)
encoded_y = encoder.transform(y_train_k)
# convert integers to dummy variables (i.e. one hot encoded)
y = keras.utils.to_categorical(encoded_y)

In [15]:
X_traink, X_testk, y_traink, y_testk = train_test_split(X_train_k, y, test_size=0.2, random_state=1)

In [16]:
# scales data between 0 and 1
X_traink = keras.utils.normalize(X_traink, axis=1)  
X_testk = keras.utils.normalize(X_testk, axis=1)

##### The Keras model-

In [17]:
def c_model():

  model = Sequential()
  model.add(Dense(2048, activation='relu', input_dim=39))
  # model.add(Dense(1024, activation='relu'))
  # model.add(Dropout(0.25))
  model.add(Dense(512, activation='relu'))
  # model.add(Dropout(0.25))
  model.add(Dense(256, activation='relu'))
  # model.add(Dropout(0.25))
  # model.add(Dense(128, activation='relu'))
  # model.add(Dropout(0.25))
  model.add(Dense(64, activation='relu'))
  # model.add(Dropout(0.5))
  model.add(Dense(30, activation='softmax'))
  
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

##### a. For epochs=15, batch_size=20000

In [18]:
model = KerasClassifier(build_fn=c_model, epochs=15, batch_size=20000)
model.fit(X_traink, y_traink)

C:\Users\User-\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




Epoch 1/15


101/101 [==============================] - 116s 1s/step - loss: 2.7444 - accuracy: 0.2212
Epoch 2/15
101/101 [==============================] - 114s 1s/step - loss: 1.7179 - accuracy: 0.4166
Epoch 3/15
101/101 [==============================] - 100s 990ms/step - loss: 1.3061 - accuracy: 0.5115
Epoch 4/15
101/101 [==============================] - 105s 1s/step - loss: 1.2456 - accuracy: 0.5307
Epoch 5/15
101/101 [==============================] - 98s 968ms/step - loss: 1.1846 - accuracy: 0.5514
Epoch 6/15
101/101 [==============================] - 131s 1s/step - loss: 0.9212 - accuracy: 0.6500
Epoch 7/15
101/101 [==============================] - 113s 1s/step - loss: 1.1601 - accuracy: 0.5767
Epoch 8/15
101/101 [==============================] - 109s 1s/step - loss: 0.8321 - accuracy: 0.6857
Epoch 9/15
101/101 [==============================] - 107s 1s/step - loss: 0.8210 - accuracy: 0.6805
Epoch 10/15
101/101 [==============================] - 108s 1s/step - loss: 0.9962

KerasClassifier(
	model=None
	build_fn=<function c_model at 0x000002010DB4EDE0>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=20000
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=15
	class_weight=None
)

In [19]:
pred=model.predict(X_testk)

26/26 [==============================] - 11s 374ms/step


In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import f1_score


print("F1 Score:", f1_score(pred,y_testk, average="micro"))
print("Accuracy:", accuracy_score(pred,y_testk))
print("Precision:",precision_score(pred,y_testk, average="micro"))
print("Recall:",recall_score(pred,y_testk,  average="micro"))


F1 Score: 0.5963146679082427
Accuracy: 0.5963146679082427
Precision: 0.5963146679082427
Recall: 0.5963146679082427


##### We can see that k=30 has decreased the accuracy of our models. Hence, k=20 is the optimum.